<a href="https://colab.research.google.com/github/yansavitski/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/01-keyword-querying-filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU elasticsearch sentence-transformers==2.2.2
from elasticsearch import Elasticsearch
# Create the client instance
client = Elasticsearch(
    cloud_id="",
    basic_auth=("elastic", "")
)
print(client.info())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00


[Quick Start](https://github.com/yansavitski/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/00-quick-start.ipynb) || **Keyword Quering Filtering** || [Hubrid search with RRF](https://github.com/yansavitski/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/02-hybrid-search-with-rrf.ipynb) || [ELSER](https://github.com/yansavitski/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/03-ELSER.ipynb)

# Keyword Quering Filtering

<a target="_blank" href="https://colab.research.google.com/github/elasticsearch-labs/colab-notebooks-examples/blob/notebooks-guides/search/01-keyword-querying-filtering.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This interactive notebook will introduce you to the Elasticsearch queries, using the official Elasticsearch Python client. Before getting start this section we highly recomented firstly finish [quick start](https://github.com/yansavitski/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/00-quick-start.ipynb)

In [ ]:
!pip install -qU elasticsearch sentence-transformers==2.2.2
from elasticsearch import Elasticsearch
# Create the client instance
client = Elasticsearch(
    cloud_id="<cloud_id>",
    basic_auth=("elastic", "<password>")
)
print(client.info())

## Querying
In the query context, a query clause answers the question “How well does this document match this query clause?” Besides deciding whether or not the document matches, the query clause also calculates a relevance score in the _score metadata field.

### Full text queries

The full text queries enable you to search analyzed text fields such as the body of an email. The query string is processed using the same analyzer that was applied to the field during indexing.

* **match**.
    The standard query for performing full text queries, including fuzzy matching and phrase or proximity queries.
* **multi-match**.
    The multi-field version of the match query.

### Compound query
* **boosting query**.
    Return documents which match a positive query, but reduce the score of documents which also match a negative query.

### Match query
Returns documents that match a provided text, number, date or boolean value. The provided text is analyzed before matching.

The match query is the standard query for performing a full-text search, including options for fuzzy matching.

[Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-query-ex-request)



In [ ]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        score = hit['_score']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nScore: {score}")
        print(pretty_output)

response = client.search(index="book_index", query={"match": {"summary": "guide"}})

pretty_response(response)